In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("combined/total_combined.csv")
# df = pd.read_csv("cumulate_combined/total_cumulate_combined.csv")
# df = pd.read_csv("exp_cumulate_combined/total_exp_cumulate_combined.csv")

df.drop(['Unnamed: 0'],axis=1, inplace = True)
print(df.shape)

# df = pd.concat([df, pd.get_dummies(df.T_ID)], axis = 1)
# data.drop(["T_ID"], axis = 1, inplace = True)

FileNotFoundError: [Errno 2] File combined/total_combined.csv does not exist: 'combined/total_combined.csv'

# 1. 승무패 불러오기

In [122]:
# year = [2016,2017,2018,2019,2020]
# team = []
# team_pitcher = []

# for i in year:
#     team_pitcher.append(pd.read_excel('excel_data/2020빅콘테스트_스포츠투아이_제공데이터_{}.xlsx'.format(i), sheet_name = '4.팀투수'))
# team_pitcher_df = pd.concat(team_pitcher, axis=0)

In [123]:
wls = team_pitcher_df[["G_ID","T_ID","VS_T_ID","WLS"]]
wls = wls.rename({'WLS':'wls'},axis=1)
wls

,G_ID,T_ID,VS_T_ID,wls
0,20160401HHLG0,LG,HH,W
1,20160401HHLG0,HH,LG,L
2,20160401HTNC0,NC,HT,W
3,20160401HTNC0,HT,NC,L
4,20160401KTSK0,SK,KT,L
...,...,...,...,...
635,20200719LTSS0,LT,SS,W
636,20200719OBHT0,HT,OB,L
637,20200719OBHT0,OB,HT,W
638,20200719WOSK0,SK,WO,W


# 2. 경기 분리

In [124]:
reverse_df = df.copy()

col = list(df.columns)
re_col = {'T_ID': 'VS_T_ID', 'VS_T_ID':'T_ID'}
for i in range(6,len(col)):
    if 'vs' in col[i]:
        re_col[col[i]] = col[i][3:]    
    else:
        re_col[col[i]] = 'vs_'+col[i]
        
reverse_df= reverse_df.rename(re_col,axis=1)

tb_sc_dict = {'T':'B', 'B':'T'}
for i in range(len(reverse_df)):
    orig = df.loc[i,"TB_SC"]
    reverse_df.loc[i,"TB_SC"] = tb_sc_dict[orig]
    

In [125]:
data = pd.concat([df, reverse_df],axis=0)
data = data.sort_values(by = ["G_ID"]).reset_index(drop = True)

# 3. 승무패 붙이기

In [126]:
fin_data = pd.merge(data, wls, how='left')

wls_dict = {'W':'L', 'D':'D', 'L':'W'}
for i in range(len(fin_data)):
    now_wls = fin_data.loc[i,'wls']
    fin_data.loc[i,'vs_wls'] = wls_dict[now_wls]

# 4. 연도 추가/상대팀 one-hot 추가

In [127]:
# 연도추가
fin_data.loc[:,'YEAR'] = fin_data.loc[:,'G_ID'].str.slice(start= 0, stop=4)

team = list(df.T_ID.unique())

# 상대팀 one-hot
# team_dict_vstid = dict()
# for i in team:
#     team_dict_vstid[i] = 'vs_'+i

# add_data = pd.concat([fin_data, pd.get_dummies(fin_data.VS_T_ID)], axis = 1)
# add_data = add_data.rename(team_dict_vstid, axis =1)

add_data = fin_data.copy()

In [128]:
add_data

,G_ID,GDAY_DS,T_ID,VS_T_ID,HEADER_NO,TB_SC,WHIP,LOB,ERA,FIP,...,vs_PF,vs_H_A,vs_WLS,vs_PITCHER_INNING,vs_PITCHER_ER,vs_HITTER_AB,vs_HITTER_HIT,wls,vs_wls,YEAR
0,20160401HHLG0,20160401,LG,HH,0,B,1.333,0.750,3.000,7.333,...,939,0,0,11.333,4.0,46.0,13.0,W,L,2016
1,20160401HHLG0,20160401,HH,LG,0,T,1.147,0.690,3.176,23.059,...,939,1,1,12.000,4.0,42.0,9.0,L,W,2016
2,20160401HTNC0,20160401,NC,HT,0,B,1.222,0.755,4.000,22.778,...,1035,0,0,8.000,5.0,34.0,8.0,W,L,2016
3,20160401HTNC0,20160401,HT,NC,0,T,1.750,0.804,5.625,38.750,...,1035,1,1,9.000,4.0,30.0,9.0,L,W,2016
4,20160401KTSK0,20160401,KT,SK,0,T,1.111,0.698,4.000,11.444,...,971,1,0,9.000,8.0,36.0,10.0,W,L,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6395,20200719LTSS0,20200719,LT,SS,0,T,1.396,0.685,4.749,22.830,...,1062,1,0,515.700,253.1,1980.4,543.4,W,L,2020
6396,20200719OBHT0,20200719,OB,HT,0,T,1.521,0.655,4.982,18.483,...,1004,1,0,490.200,237.5,1890.5,521.0,W,L,2020
6397,20200719OBHT0,20200719,HT,OB,0,B,1.369,0.681,4.360,18.168,...,1004,0,1,514.500,284.8,2060.2,623.0,L,W,2020
6398,20200719WOSK0,20200719,SK,WO,0,B,1.523,0.686,4.841,23.221,...,971,0,0,524.600,277.6,2036.4,551.5,W,L,2020


In [129]:
# add_data.to_csv("model_input_data/exp_original_data.csv", index = False)

# 5. 연도-팀별 24경기씩 통계
#### (타율/방어율/승률:sum, 나머지:mean) 

<전체 합에 대해>
- 승률: 이긴횟수/(이긴횟수+진횟수)
- 타율: 안타(HIT)/타수(AB)
- 평균자책: 자책점(ER) X 9 / 이닝(INN2/3)

In [130]:
add_col = list(add_data.columns)

default_col = ['G_ID', 'GDAY_DS', 'T_ID', 'VS_T_ID',"YEAR"]
size_col = ['wls','vs_wls']

sum_col = ['HEADER_NO',
           'PITCHER_INNING', 'PITCHER_ER', 'HITTER_AB','HITTER_HIT',  
           'vs_PITCHER_INNING', 'vs_PITCHER_ER','vs_HITTER_AB', 'vs_HITTER_HIT']
# sum_col = ['HEADER_NO',
#            'PITCHER_INNING', 'PITCHER_ER', 'HITTER_AB','HITTER_HIT',  
#            'vs_PITCHER_INNING', 'vs_PITCHER_ER','vs_HITTER_AB', 'vs_HITTER_HIT', 
#            'vs_HH', 'vs_HT','vs_KT', 'vs_LG', 'vs_LT', 'vs_NC', 'vs_OB', 'vs_SK', 'vs_SS', 'vs_WO']

# vsteam_col = ['vs_HH', 'vs_HT','vs_KT', 'vs_LG', 'vs_LT', 'vs_NC', 'vs_OB', 'vs_SK', 'vs_SS', 'vs_WO']

tmp = set(default_col)|set(size_col)|set(sum_col)
mean_col = list(set(add_col)-tmp)

In [131]:
size_df = add_data[default_col + size_col]
sum_df = add_data[default_col + sum_col]
mean_df = add_data[default_col + mean_col]

#### size

In [132]:
def make_pct(df):
    tmp = pd.DataFrame({"COUNT":df.groupby(["T_ID","wls","YEAR"]).size()}).pivot_table("COUNT",["T_ID","YEAR"],"wls").reset_index().sort_values(["YEAR","T_ID"]).reset_index(drop=True)
    tmp.fillna(0,inplace= True)

    if "L" not in list(tmp.columns):
        tmp["L"] = 0

    if "W" not in list(tmp.columns):
        tmp["W"] = 0
    
    tmp["PCT"] = tmp['W']/(tmp['W']+tmp['L'])
    return tmp

In [133]:
size_df_cal = pd.DataFrame([])

key = 24
for y in [2016,2017,2018,2019]:
    print("===",y)
    
    y_tmp = size_df[size_df["YEAR"]==str(y)]
    for t in team:
        print(t)
        tmp_df = y_tmp[y_tmp["T_ID"]==t]
        
        for k in range(6):
            tmp1 = tmp_df.iloc[key*k:key*(k+1),:]
            tmp2 = make_pct(tmp1)
            tmp2['IDX'] = k+1

            size_df_cal = pd.concat([size_df_cal, tmp2], axis = 0)

## 2020    
y_tmp = size_df[size_df["YEAR"]=='2020']
print("===",2020)

for t in team:
    print(t)

    tmp_df = y_tmp[y_tmp["T_ID"]==t]
    
    t_size = len(tmp_df)
    
    for k in range(t_size//key):
        tmp1 = tmp_df.iloc[key*k:key*(k+1),:]
        tmp2 = make_pct(tmp1)
        tmp2['IDX'] = k+1
        
        size_df_cal = pd.concat([size_df_cal, tmp2], axis = 0)
        
##

size_df_cal.drop(["W","D","L"],axis = 1, inplace = True)
        
# df16 = pd.DataFrame([])

# idx = 0
# for t in list(team["T_ID"]):
#     print(t)
#     df = tmp16_pre[tmp16_pre["T_ID"]==t]
#     for k in range(12):
#         tmp1 = df.iloc[12*k:12*(k+1),:]
#         tmp2 = make_pct(tmp1)
#         tmp2['IDX'] = k+1
        
#         df16 = pd.concat([df16, tmp2], axis = 0)
#         idx += 1

=== 2016
LG
NC
KT
WO
OB
HH
SK
LT
SS
HT
=== 2017
LG
NC
KT
WO
OB
HH
SK
LT
SS
HT
=== 2018
LG
NC
KT
WO
OB
HH
SK
LT
SS
HT
=== 2019
LG
NC
KT
WO
OB
HH
SK
LT
SS
HT
=== 2020
LG
NC
KT
WO
OB
HH
SK
LT
SS
HT


In [134]:
size_df_cal.tail(18)

,T_ID,YEAR,PCT,IDX
0,NC,2020,0.750000,1
0,NC,2020,0.625000,2
0,KT,2020,0.416667,1
0,KT,2020,0.458333,2
0,WO,2020,0.500000,1
0,WO,2020,0.750000,2
0,OB,2020,0.625000,1
0,OB,2020,0.541667,2
0,HH,2020,0.291667,1
0,HH,2020,0.208333,2


#### sum

<전체 합에 대해>
- 승률: 이긴횟수/(이긴횟수+진횟수)
- 타율: 안타(HIT)/타수(AB)
- 평균자책: 자책점(ER) X 9 / 이닝(INN2/3)

In [135]:
def make_sum(df):
    tmp = df.groupby(["T_ID","YEAR"]).sum().reset_index()
    
    # ERA
    tmp["era"] = tmp["PITCHER_ER"]*9/tmp["PITCHER_INNING"]
    tmp["vs_era"] = tmp["vs_PITCHER_ER"]*9/tmp["vs_PITCHER_INNING"]
    
    #AVG
    tmp['avg'] = tmp["HITTER_HIT"]/tmp["HITTER_AB"]
    tmp['vs_avg'] = tmp["vs_HITTER_HIT"]/tmp["vs_HITTER_AB"]
    
    re_col = ["T_ID","YEAR","HEADER_NO","era","avg","vs_era","vs_avg"]
#     re_col = ["T_ID","YEAR","HEADER_NO","era","avg","vs_era","vs_avg"] + vsteam_col
    return tmp[re_col]
    

In [136]:
sum_df_cal = pd.DataFrame([])


key = 24
for y in [2016,2017,2018,2019]:
    print("===",y)
    
    y_tmp = sum_df[sum_df["YEAR"]==str(y)]
    for t in team:
        print(t)
        tmp_df = y_tmp[y_tmp["T_ID"]==t]
        
        for k in range(6):
            tmp1 = tmp_df.iloc[key*k:key*(k+1),:]
            tmp2 = make_sum(tmp1)
            tmp2['IDX'] = k+1

            sum_df_cal = pd.concat([sum_df_cal, tmp2], axis = 0)
            
## 2020    
y_tmp = sum_df[sum_df["YEAR"]=='2020']
print("===",2020)

for t in team:
    print(t)

    tmp_df = y_tmp[y_tmp["T_ID"]==t]
    
    t_size = len(tmp_df)
    
    for k in range(t_size//key):
        tmp1 = tmp_df.iloc[key*k:key*(k+1),:]
        tmp2 = make_sum(tmp1)
        tmp2['IDX'] = k+1
        
        sum_df_cal = pd.concat([sum_df_cal, tmp2], axis = 0)
        
##

=== 2016
LG
NC
KT
WO
OB
HH
SK
LT
SS
HT
=== 2017
LG
NC
KT
WO
OB
HH
SK
LT
SS
HT
=== 2018
LG
NC
KT
WO
OB
HH
SK
LT
SS
HT
=== 2019
LG
NC
KT
WO
OB
HH
SK
LT
SS
HT
=== 2020
LG
NC
KT
WO
OB
HH
SK
LT
SS
HT


In [137]:
sum_df_cal.tail(21)

,T_ID,YEAR,HEADER_NO,era,avg,vs_era,vs_avg,IDX
0,HT,2019,0,4.717686,0.264680,4.293630,0.271027,6
0,LG,2020,3,4.579809,0.284393,4.650188,0.264471,1
0,LG,2020,6,4.331020,0.290604,4.871296,0.261766,2
0,NC,2020,0,3.386518,0.286228,5.011216,0.261289,1
0,NC,2020,3,4.261453,0.298747,5.216830,0.271454,2
0,KT,2020,0,5.560066,0.309565,4.499104,0.270775,1
0,KT,2020,6,5.831286,0.295167,4.787034,0.264500,2
0,WO,2020,3,4.058128,0.256744,4.752511,0.274313,1
0,WO,2020,3,4.347646,0.268418,4.534361,0.270553,2
0,OB,2020,0,6.085728,0.314805,4.958438,0.262127,1


#### mean

In [138]:
def make_mean(df):
    tmp = df.groupby(["T_ID","YEAR"]).mean().reset_index()
    tmp.drop(["GDAY_DS"],axis=1,inplace = True)
    return tmp

In [139]:
mean_df_cal = pd.DataFrame([])

key = 24
for y in [2016,2017,2018,2019]:
    print("===",y)
    
    y_tmp = mean_df[mean_df["YEAR"]==str(y)]
    for t in team:
        print(t)
        tmp_df = y_tmp[y_tmp["T_ID"]==t]
        
        for k in range(6):
            tmp1 = tmp_df.iloc[key*k:key*(k+1),:]
            tmp2 = make_mean(tmp1)
            tmp2['IDX'] = k+1

            mean_df_cal = pd.concat([mean_df_cal, tmp2], axis = 0)
            
## 2020    
y_tmp = mean_df[mean_df["YEAR"]=='2020']
print("===",2020)

for t in team:
    print(t)

    tmp_df = y_tmp[y_tmp["T_ID"]==t]
    
    t_size = len(tmp_df)
    
    for k in range(t_size//key):
        tmp1 = tmp_df.iloc[key*k:key*(k+1),:]
        tmp2 = make_mean(tmp1)
        tmp2['IDX'] = k+1
        
        mean_df_cal = pd.concat([mean_df_cal, tmp2], axis = 0)
        
##

=== 2016
LG
NC
KT
WO
OB
HH
SK
LT
SS
HT
=== 2017
LG
NC
KT
WO
OB
HH
SK
LT
SS
HT
=== 2018
LG
NC
KT
WO
OB
HH
SK
LT
SS
HT
=== 2019
LG
NC
KT
WO
OB
HH
SK
LT
SS
HT
=== 2020
LG
NC
KT
WO
OB
HH
SK
LT
SS
HT


In [140]:
mean_df_cal.tail(21)

,T_ID,YEAR,vs_AVG,vs_WLS,oAVG,vs_RC,HR_9,OBP,vs_SECA,vs_DER,...,SECA,vs_GPA,H_9,XR,vs_FIP,LOB,TA,P_WHIP_RT,vs_BA,IDX
0,HT,2019,0.271000,0.500000,0.280833,0.535958,0.734958,0.335000,0.302042,0.684667,...,0.282958,0.251583,9.716000,3.788167,18.408500,0.675708,0.651958,1.398333,0.271000,6
0,LG,2020,0.261083,0.333333,0.280333,0.351333,0.982000,0.361042,0.327417,0.687417,...,0.411708,0.248042,9.691833,4.803417,21.666208,0.671667,0.814708,1.063833,0.261083,1
0,LG,2020,0.260750,0.500000,0.266042,0.476250,0.921667,0.354875,0.329083,0.684375,...,0.348792,0.250292,9.102708,4.706000,22.300708,0.696167,0.772167,1.235458,0.260750,2
0,NC,2020,0.248500,0.250000,0.214958,0.280708,1.062167,0.359375,0.321000,0.688000,...,0.423958,0.241333,7.128833,5.536875,25.019542,0.828917,0.851708,0.914167,0.248500,1
0,NC,2020,0.269083,0.375000,0.247250,0.452000,1.069667,0.374875,0.319792,0.673542,...,0.411542,0.253208,8.411708,5.686542,22.264458,0.731375,0.878208,1.381583,0.269083,2
0,KT,2020,0.270167,0.583333,0.292083,0.540000,1.186500,0.350083,0.335333,0.700500,...,0.346583,0.257750,10.451833,5.165250,21.527000,0.619042,0.781500,1.762208,0.270167,1
0,KT,2020,0.264333,0.541667,0.290875,0.446375,1.123875,0.353042,0.320958,0.687750,...,0.350958,0.251208,10.382750,5.023792,22.599083,0.645500,0.785375,1.733208,0.264333,2
0,WO,2020,0.261292,0.500000,0.268375,0.377208,0.871000,0.334417,0.323417,0.697250,...,0.384042,0.249292,9.199833,4.486208,24.093958,0.731250,0.742333,1.355500,0.261292,1
0,WO,2020,0.269375,0.250000,0.274167,0.472208,1.036750,0.354083,0.329708,0.692375,...,0.388292,0.255250,9.417625,4.770708,21.988542,0.689500,0.775333,1.234500,0.269375,2
0,OB,2020,0.269125,0.375000,0.311625,0.515375,0.921833,0.364208,0.355167,0.695083,...,0.329167,0.259250,11.349250,5.266875,22.830958,0.613875,0.789542,1.741917,0.269125,1


# 6. 3개의 table 합치기

In [141]:
tmp_table = pd.merge(size_df_cal, sum_df_cal, how= 'left', left_on = ['T_ID',"YEAR","IDX"], right_on  = ['T_ID',"YEAR","IDX"])
merge_df = pd.merge(tmp_table, mean_df_cal, how = 'left', left_on = ['T_ID',"YEAR","IDX"], right_on  = ['T_ID',"YEAR","IDX"] )

In [142]:
merge_df.to_csv("model_input_data/exp_lstm_24group_1620.csv", index = False)